<a href="https://colab.research.google.com/github/MasahiroAraki/MLCourse/blob/master/14_semi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 第14章 半教師あり学習

## 例題14.3

scikit-learnのLabelPropagationでirisデータの半教師あり学習を行います。

ライブラリの読み込み

In [1]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.semi_supervised import LabelPropagation

irisデータの読み込み

In [2]:
iris = load_iris()
X = iris.data
y = iris.target

### 半教師ありデータの作成

まず、どのデータの教師信号を消すかを決めます。
[np.random.choice](https://docs.scipy.org/doc/numpy/reference/generated/numpy.random.choice.html)を使って、データのインデックスを表すndarrayから、指定された個数（全体の7割）のインデックスを、重複なしで抜き出します。

In [3]:
unlabeled_points = np.random.choice(np.arange(y.size), int((y.size)*.7), replace=False)
unlabeled_points

array([ 78,   4,  46, 126,  93, 117,  67,  98,  49,  41,  94,  90,  75,
       132,  47, 113, 147,  20,  51, 100,  79,  86, 105,  28,  52,  56,
       101,  53,  23, 118,  44,  99, 123,  15,  60, 104, 108,  31, 114,
       135, 107,   1,  19,  80, 112, 145,  70, 142,  17,  33, 139, 124,
       122,  62,  59,  88,  40, 143, 141,  96, 110,  11,  55,  13,  18,
       146,  12,   7,  50, 149,  65,  64, 133,  24,  76,  71,   8,  77,
        69,  95,  85, 120,  83,  54,  58,  26, 102,  89, 127, 131,  74,
        82, 136, 125, 144,  35,  87, 103,   9,  39,  14, 129,  30,  22,
        48])

教師ベクトルyを[np.copy](https://docs.scipy.org/doc/numpy/reference/generated/numpy.copy.html)を使って変数labelsにコピーし（代入文でコピーするとオブジェクトが共有され、labelsを変更するとyも変わってしまいます）、先ほど抜き出したインデックスに対応する値を-1に書き換えます。

In [4]:
labels = np.copy(y)
labels[unlabeled_points] = -1
labels

array([ 0, -1,  0,  0, -1,  0,  0, -1, -1, -1,  0, -1, -1, -1, -1, -1,  0,
       -1, -1, -1, -1,  0, -1, -1, -1,  0, -1,  0, -1,  0, -1, -1,  0, -1,
        0, -1,  0,  0,  0, -1, -1, -1,  0,  0, -1,  0, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1,  1, -1, -1, -1,  1, -1,  1, -1, -1,  1, -1,
        1, -1, -1, -1,  1,  1, -1, -1, -1, -1, -1, -1, -1,  1, -1, -1,  1,
       -1, -1, -1, -1, -1, -1,  1,  1, -1, -1, -1, -1,  1, -1, -1, -1, -1,
       -1, -1, -1, -1,  2, -1, -1,  2, -1,  2, -1, -1, -1,  2,  2, -1, -1,
        2, -1,  2, -1, -1, -1, -1, -1, -1,  2, -1,  2, -1, -1, -1,  2, -1,
       -1,  2,  2, -1,  2, -1, -1, -1, -1, -1, -1, -1,  2, -1])

これを教師ベクトルとして、[LabelPropagation](http://scikit-learn.org/stable/modules/generated/sklearn.semi_supervised.LabelPropagation.html)で半教師あり学習を行います。

In [5]:
lp = LabelPropagation(max_iter=10000)
lp.fit(X, labels)

LabelPropagation(gamma=20, kernel='rbf', max_iter=10000, n_jobs=None,
                 n_neighbors=7, tol=0.001)

正解なしデータの割り当て結果を表示します。

In [6]:
lp.score(X[unlabeled_points], y[unlabeled_points])

0.9809523809523809

正解付きデータを5%, 10%, 20%, 30%と変えて、それぞれ100回ずつ学習を試みて性能を評価します。

In [7]:
labeled_percent = [0.05, 0.1, 0.2, 0.3]
num = y.size
for labeled in labeled_percent :
    score = 0
    for i in range(100):
        unlabeled_points = np.random.choice(np.arange(num),int(num-num*labeled), replace=False)
        labels = np.copy(y)
        labels[unlabeled_points] = -1
        lp.fit(X, labels)
        score += lp.score(X[unlabeled_points], y[unlabeled_points])
    print("{0}{1:4.1f}{2}{3:6.3f}".format("labeled:", labeled*100, "%, score=", score/100))        

labeled: 5.0%, score= 0.850
labeled:10.0%, score= 0.918
labeled:20.0%, score= 0.954
labeled:30.0%, score= 0.954


## 課題

使用するデータをbreast_cancerに変更して、同様の手順で半教師あり学習を行ってください。その際、以下の点に注意してください。

* 学習時にエラーが出てしまう場合は、irisデータとbreast_cancerデータの両方のデータを見て、何が違うかを考えてみてください。
* 低い性能しか出ないときは、LabelPropagationのハイパーパラメータ調整を試みてください。
